In [1]:
import numpy as np
import math
import json
def generateVocab():
    with open('training.txt', 'r') as data:
            lines = data.read().splitlines() 
    occurences = {}
    doccurence = {}
    appearances = {}
    uniquewords = {}
    for x in range(len(lines)):
        appearances[x] = {}
        currentLine = lines[x].split(' ')
        uniquewords[x] = list(filter(lambda x: x.isalnum(), list(set(currentLine))))
        alreadyin = {}
        for y in range(1000):
            if currentLine[y].isalnum():
                try:
                    z = alreadyin[currentLine[y]]
                except KeyError:
                    alreadyin[currentLine[y]] = True
                    try:
                        doccurence[currentLine[y]] += 1
                    except KeyError:
                        doccurence[currentLine[y]] = 1
                        
                try:
                    occurences[currentLine[y]] += 1
                except KeyError: 
                    occurences[currentLine[y]] = 1
                finally:
                    try: 
                        appearances[x][currentLine[y]] += 1
                    except KeyError:
                        appearances[x][currentLine[y]] = 1
    occurences = {k: v for k, v in sorted(occurences.items(), key=lambda item: item[1], reverse=True)}
    with open('unique_words_per_doc.txt', 'w') as fout:
        fout.write(json.dumps(uniquewords))
    with open('appearances_in_doc.txt', 'w') as fout:
        fout.write(json.dumps(appearances))
    with open('occurences.txt', 'w') as fout:
        fout.write(json.dumps(occurences))
    with open('doccurence.txt', 'w') as fout:
        fout.write(json.dumps(doccurence))
    return

In [2]:
generateVocab()

In [3]:
def Bag_of_Word_Feature_Extractor(string):
    occurences = generateVocab()
    bag = {} 
    counter = 0
    for key in occurences:
        bag[key] = counter
        counter += 1
    #bag maps word to index
    string = string.split(' ')
    ans = np.zeros(len(occurences.keys()))
    for x in range(len(string)):
        ans[bag[string[x]]] += 1
    return ans
    

In [4]:
def TF_IDF_Feature_Extractor(string):
    with open('occurences.txt', 'r') as fin:
        occurences = json.loads(fin.read())
    bag = {} 
    counter = 0
    for key in occurences:
        bag[key] = counter
        counter += 1
    #bag maps word to index
    string = string.split(' ')
    ans = np.zeros(len(occurences.keys()))
    uniqueWords = []
    for z in range(len(string)):
        if string[z] not in uniqueWords:
            uniqueWords.append(string[z])
    with open('training.txt', 'r') as data:
        lines = data.read().splitlines() 
    totalDocs = len(lines);
    for x in range(len(uniqueWords)):
        cAppearances = 0
        for y in range(len(string)):
            if string[y] == uniqueWords[x]:
                cAppearances += 1
        #calculate appearances in document
        dApperances = 0;
        for y in range(totalDocs):
            currentLine = lines[y].split(' ')
            if uniqueWords[x] in currentLine:
                dApperances += 1
        #calculate appearances in all documents
        print(uniqueWords[x],  (cAppearances/len(string)) * math.log(totalDocs/dApperances))
        if (dApperances == 0):
            continue
        else:
            ans[bag[uniqueWords[x]]] = (cAppearances/len(string)) * math.log(totalDocs/dApperances)
    return ans
    

In [5]:
def generateFeatureVectors():
    with open('occurences.txt', 'r') as fin:
        occurences = json.loads(fin.read())
    bag = {} 
    counter = 0
    for key in occurences:
        bag[key] = counter
        counter += 1
    result = ()
    kL = len(occurences.keys())
    with open('training.txt', 'r') as data:
        lines = data.read().splitlines() 
    with open('doccurence.txt', 'r') as fin:
        docAp = json.loads(fin.read())
    with open('unique_words_per_doc.txt', 'r') as fin:
        allUniqueWords = json.loads(fin.read())
    with open('appearances_in_doc.txt', 'r') as fin:
        cap = json.loads(fin.read())
    totalDocs = len(lines);
    for i, y in enumerate(lines): 
        string = y.split(' ')
        ans = np.zeros(kL)
        uniqueWords = allUniqueWords[str(i)]
        for x in range(len(uniqueWords)):
            cAppearances = cap[str(i)][uniqueWords[x]]
            #calculate appearances in document
            dApperances = docAp[uniqueWords[x]]
            #print(uniqueWords[x],  (cAppearances/len(string)) * math.log(totalDocs/dApperances))
            ans[bag[uniqueWords[x]]] = (cAppearances/len(string)) * math.log(totalDocs/dApperances)
        result = result + (ans,)
    ans = np.row_stack(result)
    np.save('feats.npy', ans)
        
        
    

In [6]:
generateFeatureVectors()

In [7]:
a = np.load('feats.npy')

In [10]:
print(a[0].shape)
for x in a[0]:
    y = x * 10000
    if y != 0:
        
        print("{0:.10f}".format(x))

(10000,)
0.0001416432
0.0000519740
0.0000449775
0.0000149925
0.0000109908
0.0001979638
0.0000773493
0.0000293579
0.0000589836
0.0006448664
0.0001311491
0.0000844236
0.0000046675
0.0004580243
0.0023915253
0.0000625870
0.0000174678
0.0000249510
0.0000905912
0.0000888935
0.0001524059
0.0001342832
0.0001335710
0.0000508916
0.0002599256
0.0005708749
0.0008282094
0.0001979638
0.0003029039
0.0001345404
0.0003492083
0.0009387477
0.0003560212
0.0001753025
0.0030470668
0.0004360039
0.0002045993
0.0005779840
0.0005396858
0.0001383577
0.0001205416
0.0008469785
0.0005213486
0.0009050248
0.0002200111
0.0010305996
0.0006600334
0.0004659219
0.0028748413
0.0005474492
0.0004448012
0.0004095657
0.0014009577
0.0015965948
0.0018245920
0.0003240220
0.0007126373
0.0012956720
0.0013164053
0.0003921567
0.0010462081
0.0004818643
0.0010051507
0.0004563021
0.0012559614
0.0008761338
0.0005453319
0.0005390291
0.0019339354
0.0004610455
0.0005009246
0.0010757747
0.0005557323
0.0007472082
0.0011525409
0.0011466209
0.0

In [10]:
with open('occurences.txt', 'r') as fin:
    occurences = json.loads(fin.read())
print(len(occurences.keys()))

10000
